In [43]:
import pandas as pd
# 출력물의 생략없이 모두 출력
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import numpy as np
np.set_printoptions(threshold=np.inf)

import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import plotly.express as px

import re
import os 
import pickle

import re
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

import requests
import json

# 한글 폰트 사용을 위해서 세팅
from matplotlib import font_manager, rc
# font_path = "D:/project/fonts/NanumGothic.ttf"
# font_path = 'G:/다른 컴퓨터/My_desktop/project/fonts/NanumGothic.ttf'
# 한글 폰트 설정
font_path = 'G:/다른 컴퓨터/내 컴퓨터/project/fonts/NanumGothic.ttf'
# font_path = 'C:/project/fonts/NanumGothic.ttf'/
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

In [44]:
# 파일 경로 설정
file_path = r'G:/다른 컴퓨터/내 컴퓨터/project/23년 공익데이터 실험실 2기/데이터전처리/0825_통합 데이터셋_merge_최종3.xlsx'
# file_path = 'C:/project/23년 공익데이터 실험실 2기/데이터전처리/공익중개사_최종 데이터_0824.xlsx'
# CSV 파일 불러오기
df = pd.read_excel(file_path)

# 데이터프레임 확인하기
print(df.head())


  작성자  구역  No                                                 비고  \
0  쉬라   4   1                            소유자와 공유자 같은 주소(부부로 추정됨)   
1  쉬라   4   3                            소유자와 공유자 같은 주소(부부로 추정됨)   
2  쉬라   4   4                                                NaN   
3  쉬라   4   6  근저당권이 2개 설정되어 있음. 추가 근저당 채권은 60,000,000원, 채무자 ...   
4  쉬라   4   7                                                NaN   

                    지번주소                  도로명주소 대장구분        주용도  \
0   서울특별시 관악구 신림동 1552-8  서울특별시 관악구 신림로17길 75-6   일반       단독주택   
1   서울특별시 관악구 신림동 257-32    서울특별시 관악구 대학10길 9-3   일반       단독주택   
2   서울특별시 관악구 신림동 253-28     서울특별시 관악구 대학12길 11   일반       단독주택   
3   서울특별시 관악구 신림동 255-40     서울특별시 관악구 대학12길 20   일반  제2종근린생활시설   
4  서울특별시 관악구 신림동 255-248     서울특별시 관악구 대학12길 21   일반     근린생활시설   

            기타용도   건물명  대지면적(㎡)  건축면적(㎡)  건폐율(%)  연면적(㎡)  용적률산정연면적(㎡)  용적률(%)  \
0        단독주택 외1  정보없음   194.38   116.14   59.75  448.67       353.07  181.64   
1  다가구용단독주택(5가구)  정보없음   1

In [45]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 82 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   작성자              1124 non-null   object 
 1   구역               1124 non-null   int64  
 2   No               1124 non-null   int64  
 3   비고               221 non-null    object 
 4   지번주소             1124 non-null   object 
 5   도로명주소            1124 non-null   object 
 6   대장구분             1124 non-null   object 
 7   주용도              1124 non-null   object 
 8   기타용도             1124 non-null   object 
 9   건물명              1124 non-null   object 
 10  대지면적(㎡)          1124 non-null   float64
 11  건축면적(㎡)          1124 non-null   float64
 12  건폐율(%)           1124 non-null   float64
 13  연면적(㎡)           1124 non-null   float64
 14  용적률산정연면적(㎡)      1124 non-null   float64
 15  용적률(%)           1124 non-null   float64
 16  구조               1124 non-null   object 
 17  지붕구조          

## 위경도 추출

In [46]:
df_kakao = df.copy()
df_kakao.reset_index(drop=True, inplace=True)

df_kakao['위도'] = np.nan
df_kakao['경도'] = np.nan

df_kakao.head(2)

,작성자,구역,No,비고,지번주소,도로명주소,대장구분,주용도,기타용도,건물명,대지면적(㎡),건축면적(㎡),건폐율(%),연면적(㎡),용적률산정연면적(㎡),용적률(%),구조,지붕구조,세대수(세대),가구수(가구),높이(m),지상층수,지하층수,총동연면적(㎡),허가일,사용승인일,옥탑여부,가등기,가등기말소,신탁,신탁말소,압류,압류말소,가압류,가압류말소,경매개시,경매말소,소유권이전등기가처분,소유권이전등기가처분말소,근저당권설정,채권총액,제3금융권채권여부,개인채권여부,채권최고액,채권최고액채무자이름,채권최고액채무자유형,채권최고액\n근저당권자,현재전세권,전세금(원),말소포함임차권유무,현재집주인과임차권의관련성,임차권최종말소여부,매매가(원),집지분소유자수(명),소유자(공유자)-1유형,소유자(공유자)-1이름,소유자(공유자)-1지분(%),소유자(공유자)-1주소,소유자(공유자)-1실거주,소유자(공유자)1생년월일,소유자(공유자)1나이,2번째공유자유형,2번째공유자이름,2번째공유자지분(%),2번째공유자주소,2번째공유자실거주,2번째공유자생년월일,2번째공유자나이,3번째공유자유형,3번째공유자이름,3번째공유자지분(%),3번째공유자주소,3번째공유자실거주,3번째공유자생년월일,3번째공유자나이,4번째공유자유형,4번째공유자이름,4번째공유자4지분(%),소유자(공유자)-4주소,소유자(공유자)-4실거주,소유자(공유자)-4생년월일,소유자(공유자)-4나이,위도,경도
0,쉬라,4,1,소유자와 공유자 같은 주소(부부로 추정됨),서울특별시 관악구 신림동 1552-8,서울특별시 관악구 신림로17길 75-6,일반,단독주택,단독주택 외1,정보없음,194.38,116.14,59.75,448.67,353.07,181.64,철근콘크리트구조,(철근)콘크리트,0,1,13.5,4,1,448.67,20170823.0,20180409,O,0,N,0,N,0,N,0,N,0,N,0,N,O,1040000000,X,X,1040000000,이동현,공유자,영농농업협동조합,X,0,X,해당없음,해당없음,0,2,사람,이동현,70.0,서울특별시 관악구 신림로17길 75-6,X,850123,38,사람,유리나라,30.0,서울특별시 관악구 호암로22가길 28(신림동),X,840208.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,쉬라,4,3,소유자와 공유자 같은 주소(부부로 추정됨),서울특별시 관악구 신림동 257-32,서울특별시 관악구 대학10길 9-3,일반,단독주택,다가구용단독주택(5가구),정보없음,155.55,92.55,59.50,232.99,154.29,99.45,벽돌구조,(철근)콘크리트,0,5,7.8,2,1,232.99,19921222.0,19930529,X,0,N,0,N,0,N,0,N,0,N,0,N,X,0,X,X,0,해당없음,해당없음,해당없음,X,0,X,해당없음,해당없음,0,2,사람,황창노,50.0,서울특별시 관악구 신림동 251-86,O,670120,56,사람,박형숙,50.0,서울 관악구 신림동 251-86,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [47]:
rest_api_key = "08d64ac0652c8f9e1c4ea45118910c53"

In [48]:
class KakaoLocalAPI:
    """
    Kakao Local API Controller
    """
    def __init__(self, rest_api_key):
        """
        Rest API key 초기화 및 기능 별 url 설정
        """
        
        # REST API key 설정
        self.rest_api_key = rest_api_key
        self.headers = {"Authorization": "KakaoAK {}".format(rest_api_key)}
        
        # 01 주소검색
        self.URL_01 = "https://dapi.kakao.com/v2/local/search/address.json"
        
    def search_address(self, query, analyze_type = None, page = None, size = None):
        """
        01 주소 검색
        """
        params = {"query": f"{query}"}
    
        if analyze_type != None:
            params["analyze_type"]=f"{analyze_type}"
    
        elif page != None:
            params['page'] = f"{page}"
    
        elif size != None:
            params['size'] = f"{size}"
    
        res = requests.get(self.URL_01, headers = self.headers, params = params)
        document = json.loads(res.text)
    
        return document

In [49]:
kakao = KakaoLocalAPI(rest_api_key)

In [50]:
add_list = df_kakao['도로명주소']
add_list

0         서울특별시 관악구 신림로17길 75-6
1           서울특별시 관악구 대학10길 9-3
2            서울특별시 관악구 대학12길 11
3            서울특별시 관악구 대학12길 20
4            서울특별시 관악구 대학12길 21
5             서울특별시 관악구 대학14길 6
6            서울특별시 관악구 대학14길 10
7          서울특별시 관악구 대학14길 20-6
8              서울특별시 관악구 대학길 78
9            서울특별시 관악구 대학길 92-1
10           서울특별시 관악구 대학길 92-5
11           서울특별시 관악구 대학길 92-7
12          서울특별시 관악구 신림로17길 81
13          서울특별시 관악구 신림로17길 83
14          서울특별시 관악구 대학10길 9-6
15          서울특별시 관악구 대학10길 9-8
16           서울특별시 관악구 대학10길 15
17           서울특별시 관악구 대학10길 15
18           서울특별시 관악구 대학10길 15
19           서울특별시 관악구 대학10길 15
20           서울특별시 관악구 대학12길 14
21           서울특별시 관악구 대학12길 14
22           서울특별시 관악구 대학12길 14
23           서울특별시 관악구 대학12길 14
24           서울특별시 관악구 대학12길 14
25         서울특별시 관악구 대학14길 20-3
26         서울특별시 관악구 대학14길 20-5
27           서울특별시 관악구 대학14길 22
28           서울특별시 관악구 대학14길 24
29             서울특별시 관악구 대학길 74
30             서울특별시 관악구 대학길 74
31      

In [51]:
# 호출 함수 확인
kakao.search_address(add_list[1]) 

{'documents': [{'address': {'address_name': '서울 관악구 신림동 257-32',
    'b_code': '1162010200',
    'h_code': '1162073500',
    'main_address_no': '257',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '관악구',
    'region_3depth_h_name': '대학동',
    'region_3depth_name': '신림동',
    'sub_address_no': '32',
    'x': '126.937175496067',
    'y': '37.467389191365'},
   'address_name': '서울 관악구 대학10길 9-3',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 관악구 대학10길 9-3',
    'building_name': '',
    'main_building_no': '9',
    'region_1depth_name': '서울',
    'region_2depth_name': '관악구',
    'region_3depth_name': '신림동',
    'road_name': '대학10길',
    'sub_building_no': '3',
    'underground_yn': 'N',
    'x': '126.937175496067',
    'y': '37.467389191365',
    'zone_no': '08819'},
   'x': '126.937175496067',
   'y': '37.467389191365'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

In [52]:
for i in tqdm(range(len(add_list))):
    try :
        df_kakao['경도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['x']
        df_kakao['위도'].iloc[i] = kakao.search_address(add_list[i])['documents'][0]['y']
    except:
        pass
    
print(df_kakao.isnull().sum())

100%|██████████| 1124/1124 [02:49<00:00,  6.62it/s]

작성자                   0
구역                    0
No                    0
비고                  903
지번주소                  0
도로명주소                 0
대장구분                  0
주용도                   0
기타용도                  0
건물명                   0
대지면적(㎡)               0
건축면적(㎡)               0
건폐율(%)                0
연면적(㎡)                0
용적률산정연면적(㎡)           0
용적률(%)                0
구조                    0
지붕구조                  0
세대수(세대)               0
가구수(가구)               0
높이(m)                 0
지상층수                  0
지하층수                  0
총동연면적(㎡)              0
허가일                   9
사용승인일                 0
옥탑여부                  0
가등기                   0
가등기말소                 0
신탁                    0
신탁말소                  0
압류                    0
압류말소                  0
가압류                   0
가압류말소                 0
경매개시                  0
경매말소                  0
소유권이전등기가처분            0
소유권이전등기가처분말소          0
근저당권설정                0
채권총액                  0
제3금융권채권여부       

9건이 진행안됨.
변환 실패한 9건 체크

In [53]:
result = df_kakao[df_kakao['위도'].isnull() & df_kakao['경도'].isnull()]
result.index

Int64Index([121, 122, 444, 466, 467, 533, 755, 891, 998], dtype='int64')

121, 122, 444, 466, 467, 533, 755, 891, 998 구글맵으로 수동 입력

In [54]:
df_kakao.to_csv("G:/다른 컴퓨터/내 컴퓨터/project/23년 공익데이터 실험실 2기/데이터전처리/0831_위경도추출.csv", index=False, encoding='cp949')

In [55]:
a= result[['지번주소', '도로명주소']]
a

,지번주소,도로명주소
121,서울특별시 관악구 신림동 1562-10,서울특별시 관악구 신림로21길 79
122,서울특별시 관악구 신림동 1562-11,서울특별시 관악구 신림로21길 81
444,서울특별시 관악구 신림동 255-36 서울드림빌,서울특별시 관악구 대학로 14길 13
466,서울특별시 관악구 신림동 255-245,서울특별시 관악구 대학17길 97
467,서울특별시 관악구 신림동 255-246,서울특별시 관악구 대학17길 99
533,서울특별시 관악구 신림동 251-164,서울특별시 관악구 신림로11길 76
755,서울특별시 관악구 신림동 246-33,서울특별시 관악구 대학7길 39-4
891,서울특별시 관악구 신림동 251-109,서울특별시 관악구 신림로11길 109-3
998,서울특별시 관악구 신림동 251-119,서울특별시 관악구 신림로11길 113


## 실거주자 위치

In [56]:
df_lo = df_kakao.copy()
df_lo.reset_index(drop=True, inplace=True)

df_lo['소유자1위도'] = np.nan
df_lo['소유자1경도'] = np.nan

df_lo.head(2)

,작성자,구역,No,비고,지번주소,도로명주소,대장구분,주용도,기타용도,건물명,대지면적(㎡),건축면적(㎡),건폐율(%),연면적(㎡),용적률산정연면적(㎡),용적률(%),구조,지붕구조,세대수(세대),가구수(가구),높이(m),지상층수,지하층수,총동연면적(㎡),허가일,사용승인일,옥탑여부,가등기,가등기말소,신탁,신탁말소,압류,압류말소,가압류,가압류말소,경매개시,경매말소,소유권이전등기가처분,소유권이전등기가처분말소,근저당권설정,채권총액,제3금융권채권여부,개인채권여부,채권최고액,채권최고액채무자이름,채권최고액채무자유형,채권최고액\n근저당권자,현재전세권,전세금(원),말소포함임차권유무,현재집주인과임차권의관련성,임차권최종말소여부,매매가(원),집지분소유자수(명),소유자(공유자)-1유형,소유자(공유자)-1이름,소유자(공유자)-1지분(%),소유자(공유자)-1주소,소유자(공유자)-1실거주,소유자(공유자)1생년월일,소유자(공유자)1나이,2번째공유자유형,2번째공유자이름,2번째공유자지분(%),2번째공유자주소,2번째공유자실거주,2번째공유자생년월일,2번째공유자나이,3번째공유자유형,3번째공유자이름,3번째공유자지분(%),3번째공유자주소,3번째공유자실거주,3번째공유자생년월일,3번째공유자나이,4번째공유자유형,4번째공유자이름,4번째공유자4지분(%),소유자(공유자)-4주소,소유자(공유자)-4실거주,소유자(공유자)-4생년월일,소유자(공유자)-4나이,위도,경도,소유자1위도,소유자1경도
0,쉬라,4,1,소유자와 공유자 같은 주소(부부로 추정됨),서울특별시 관악구 신림동 1552-8,서울특별시 관악구 신림로17길 75-6,일반,단독주택,단독주택 외1,정보없음,194.38,116.14,59.75,448.67,353.07,181.64,철근콘크리트구조,(철근)콘크리트,0,1,13.5,4,1,448.67,20170823.0,20180409,O,0,N,0,N,0,N,0,N,0,N,0,N,O,1040000000,X,X,1040000000,이동현,공유자,영농농업협동조합,X,0,X,해당없음,해당없음,0,2,사람,이동현,70.0,서울특별시 관악구 신림로17길 75-6,X,850123,38,사람,유리나라,30.0,서울특별시 관악구 호암로22가길 28(신림동),X,840208.0,39.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.467258268691,126.936658333376,NaN,NaN
1,쉬라,4,3,소유자와 공유자 같은 주소(부부로 추정됨),서울특별시 관악구 신림동 257-32,서울특별시 관악구 대학10길 9-3,일반,단독주택,다가구용단독주택(5가구),정보없음,155.55,92.55,59.50,232.99,154.29,99.45,벽돌구조,(철근)콘크리트,0,5,7.8,2,1,232.99,19921222.0,19930529,X,0,N,0,N,0,N,0,N,0,N,0,N,X,0,X,X,0,해당없음,해당없음,해당없음,X,0,X,해당없음,해당없음,0,2,사람,황창노,50.0,서울특별시 관악구 신림동 251-86,O,670120,56,사람,박형숙,50.0,서울 관악구 신림동 251-86,O,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.467389191365,126.937175496067,NaN,NaN


In [57]:
add_list2 = df_lo['소유자(공유자)-1주소']
add_list2

0                                   서울특별시 관악구 신림로17길 75-6
1                                    서울특별시 관악구 신림동 251-86
2                                 서울특별시 관악구 서림1길 21 (신림동)
3                                    서울특별시 관악구 신림동 255-40
4                                     서울특별시 관악구 신림동 361-8
5                                  서울특별시 관악구 대학14길 6(신림동)
6                                 서울특별시 관악구 대학14길 10(신림동)
7                               서울특별시 관악구 대학14길 20-6(신림동)
8                                                      법인
9                                서울특별시 관악구 대학길 92-1 (신림동)
10               서울특별시 송파구 위례성대로 176, 5동 505호(오금동, 대림아파트)
11                                   서울특별시 관악구 신림동 253-11
12                                  서울특별시 관악구 신림동 1552-13
13                                  서울특별시 관악구 신림동 254-261
14                               서울특별시 관악구 대학10길 9-6(신림동)
15               서울특별시 관악구 대학길 168, 에이동 201호 (신림동,동일용화빌라)
16                          서울특별시 관악구 신림동 1552-2 동궁빌라 101
17            

In [58]:
for i in tqdm(range(len(add_list))):
    try :
        df_lo['소유자1경도'].iloc[i] = kakao.search_address(add_list2[i])['documents'][0]['x']
        df_lo['소유자1위도'].iloc[i] = kakao.search_address(add_list2[i])['documents'][0]['y']
    except:
        pass
    
print(df_lo.isnull().sum())

100%|██████████| 1124/1124 [02:54<00:00,  6.43it/s]

작성자                   0
구역                    0
No                    0
비고                  903
지번주소                  0
도로명주소                 0
대장구분                  0
주용도                   0
기타용도                  0
건물명                   0
대지면적(㎡)               0
건축면적(㎡)               0
건폐율(%)                0
연면적(㎡)                0
용적률산정연면적(㎡)           0
용적률(%)                0
구조                    0
지붕구조                  0
세대수(세대)               0
가구수(가구)               0
높이(m)                 0
지상층수                  0
지하층수                  0
총동연면적(㎡)              0
허가일                   9
사용승인일                 0
옥탑여부                  0
가등기                   0
가등기말소                 0
신탁                    0
신탁말소                  0
압류                    0
압류말소                  0
가압류                   0
가압류말소                 0
경매개시                  0
경매말소                  0
소유권이전등기가처분            0
소유권이전등기가처분말소          0
근저당권설정                0
채권총액                  0
제3금융권채권여부       

In [69]:
result = df_lo[df_lo['소유자1위도'].isnull() & df_lo['소유자1경도'].isnull()]
result.index

Int64Index([   8,   40,   66,  106,  115,  130,  133,  139,  147,  189,
            ...
            1027, 1028, 1030, 1031, 1032, 1049, 1079, 1109, 1112, 1123],
           dtype='int64', length=120)

In [73]:
result_person = result[result['소유자(공유자)-1유형'] != '법인']
result_person.index

Int64Index([  40,  130,  139,  147,  189,  211,  241,  318,  390,  391,  469,
             483,  515,  519,  520,  533,  582,  621,  673,  676,  731,  762,
             774,  785,  810,  812,  835,  840,  847,  872,  930,  934, 1079,
            1123],
           dtype='int64')

34개에 대해 수기입력 진행

In [63]:
df_lo.to_csv("G:/다른 컴퓨터/내 컴퓨터/project/23년 공익데이터 실험실 2기/데이터전처리/0831_위경도추출_소유자추출.csv", index=False, encoding='cp949')